# goal: create AI agents to research and write an article

## notes and questions
- q. do we take the "multiple strings instead of triple quote docstrings to avoid newling characters" approach in our work code? the reasoning given in the lesson is that the approach lets us avoid adding whitespaces and newline characters, making the text better formatted to be passed to the LLM.
- q. notice how "researcher" is not one of the agents in this mix, nor are we giving it any tools that let it search the web. does that mean that the agent relies on the LLM it is using as its sole source of information? or do our agents have access to the internet despite us not explicitly providing them with web search tools?
- q. what is the point in splitting agents and tasks?
- llms perform better when roleplaying. make sure to provide an agent with a bare minimum of these three attributes: `role`, `goal`, `backstory`.
- provide your tasks a bare minimum of these three attributes: `description`, `expected_output` and `agent`.

## main code

In [1]:
# warning control
import warnings
warnings.filterwarnings('ignore')

In [9]:
from crewai import Agent, Task, Crew
from IPython.display import Markdown

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ['OPENAI_MODEL_NAME'] = 'gpt-3.5-turbo'

In [5]:
# agents

# planner
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",  # we interpolate variables in prompts using {}
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True  # verbose=True gives us a log of sorts on the agent's execution and its "inner thoughts"
)


# writer
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provided by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provided by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)


# editor
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

In [6]:
# tasks

# research and content outlining/planning
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)


# article writing
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)


# proofreading and editing
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [8]:
# assembling the crew

crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],  # the output of one task will be the input of the next, so we want execution of tasks in a specific order - so the order in which you pass them in this list matters
    verbose=2  # verbose=2 allows us to see all logs of the execution
)

2025-03-02 13:15:12,431 - 8469596224 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [10]:
# running the crew

result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})  # run the crew and save the result

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
Artificial Intelligence (AI) has been a hot topic in recent years, with advancements and innovations shaping the way we live and work. In this blog article, we will explore the latest trends, key players, and noteworthy news in the field of AI.

Key Points:
1. Latest Trends:
- Machine Learning: The use of algorithms and statistical models to enable computers to learn and make decisions without explicit programming.
- Natural

In [11]:
Markdown(result)  # display the result

# The Future of Artificial Intelligence: Trends, Players, and News

Artificial Intelligence (AI) has undeniably transformed various industries in recent years, pushing the boundaries of what was once thought possible. In this blog post, we will explore the latest trends, key players, and noteworthy news in the ever-evolving world of AI.

## Latest Trends

**Machine Learning:** One of the most impactful trends in AI is the widespread use of machine learning algorithms, enabling computers to learn and make decisions independently. This has led to the creation of sophisticated systems capable of processing massive amounts of data and deriving valuable insights.

**Natural Language Processing:** Another significant trend is the advancements in natural language processing, which empower machines to comprehend and generate human language. This progress has paved the way for the proliferation of chatbots and virtual assistants, enhancing communication between humans and machines.

**Robotics:** AI-driven robots are now being employed across various sectors, from manufacturing to healthcare, to execute tasks autonomously. This trend is reshaping industries by enhancing efficiency and accuracy in diverse processes.

**Ethical AI:** With the rapid progress of AI technology, the conversation around ethics and responsible AI deployment has become crucial. It is essential to ensure that AI systems are developed and utilized ethically, considering the potential societal and individual impacts.

## Key Players

**Google:** Google stands out as a leading player in the AI realm, known for its deep learning techniques and AI-driven products like Google Search and Google Assistant. The company remains at the forefront of AI research and development.

**IBM:** IBM offers a wide array of AI solutions, with Watson being a standout cognitive computing system used across industries for data analysis and decision-making. IBM's dedication to AI innovation has cemented its position as a key player in the industry.

**Amazon:** Leveraging AI in its e-commerce platform, recommendation systems, and cloud services through Amazon Web Services, Amazon has elevated customer experiences and streamlined processes through AI technology.

**Microsoft:** Microsoft is a pioneer in AI development, with notable AI technologies like Azure Cognitive Services and Cortana. The company's commitment to advancing AI solutions for both businesses and consumers has established it as a major player in the field.

## Noteworthy News

**OpenAI:** OpenAI, a research organization focused on developing AI in a safe and beneficial manner, recently introduced GPT-3, a language model capable of generating human-like text. This breakthrough has significant implications for natural language processing and AI-generated content.

**Tesla:** A frontrunner in autonomous driving technology, Tesla continues to push the boundaries of AI in the automotive industry. The company's innovative approach to AI-powered vehicles underscores the potential for AI to transform transportation.

**DeepMind:** A subsidiary of Google, DeepMind is renowned for its groundbreaking AI research, particularly with AlphaGo, an AI system that defeated the world champion in the game of Go. DeepMind's contributions to AI advancement have played a pivotal role in shaping the future of intelligent systems.

In conclusion, the landscape of Artificial Intelligence is rapidly evolving, with emerging trends, key industry players, and groundbreaking news shaping its trajectory. By staying abreast of the latest developments in AI, individuals and businesses can harness the power of AI to foster innovation and make well-informed decisions for a future where AI's role in society is paramount.

## scratchpad

- checking to see if my agents are accessing the internet or relying on the information stored in the model.
- I am writing this on March 2, 2025. The Trump-Zelenskyy Oval Office shitshow happened less than two days ago, so the only way the output will have anything concrete is if my agents are accessing the internet.
- based on the vagueness of the output, I am almost certain that my agents did not access the internet and pulled their output out of their ass.
- once I incorportate tools, I am going to create the same agents and give them access to Serper and Exa and see if the result is more concrete.

In [12]:
result0 = crew.kickoff(inputs={"topic": "The Trump-Zelenskyy Oval Office Meeting on February 28, 2025."})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on The Trump-Zelenskyy Oval Office Meeting on February 28, 2025..
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now have all the information needed to create a comprehensive content plan on The Trump-Zelenskyy Oval Office Meeting on February 28, 2025.

Final Answer:

Content Plan:
Title: The Trump-Zelenskyy Oval Office Meeting: A Closer Look at the February 28, 2025 Encounter

Outline:
I. Introduction
- Brief overview of the meeting between former President Trump and President Zelenskyy on February 28, 2025
- Importance of the meeting and its impact on international relations
- Preview of key points to be discussed in the ar

In [13]:
Markdown(result0)

# The Trump-Zelenskyy Oval Office Meeting: A Closer Look at the February 28, 2025 Encounter

## Introduction

The meeting between former President Trump and President Zelenskyy on February 28, 2025, was a significant event that captured the attention of the international community. The importance of this meeting cannot be understated, as it had a direct impact on the diplomatic relations between the United States and Ukraine. In this article, we will delve into the key points discussed during the meeting and analyze its implications on global politics.

## Latest Trends and Key Players

The latest trends surrounding the Trump-Zelenskyy Oval Office meeting indicate a renewed focus on international diplomacy and cooperation. Key players involved in the meeting include former President Trump and President Zelenskyy, along with their respective diplomatic teams. Noteworthy news related to the meeting includes discussions on trade agreements, security cooperation, and regional stability.

## Target Audience

The target audience for this article includes political enthusiasts, international relations scholars, and news junkies who are interested in understanding diplomatic relations and analyzing world events. This audience is seeking reliable information and in-depth analysis of the Trump-Zelenskyy meeting, as well as unbiased reporting on the implications of this encounter on global politics.

## Detailed Content

During the Trump-Zelenskyy Oval Office meeting, key points of discussion included bilateral trade agreements, security cooperation, and regional stability in Eastern Europe. The implications of this meeting on US-Ukraine relations are significant, as it sets the tone for future diplomatic engagements between the two countries. Quotes from relevant officials and experts provide added insight into the outcomes of the meeting and its impact on international relations.

## SEO Keywords

- Trump-Zelenskyy meeting
- Oval Office encounter
- US-Ukraine relations
- International diplomacy
- Political news update

## Resources

Official statements from the White House and Ukrainian government, along with expert analysis from political commentators and foreign policy experts, offer valuable insights into the Trump-Zelenskyy Oval Office meeting. News articles from reputable sources covering the meeting provide additional context and background information for readers interested in a comprehensive overview of this diplomatic encounter.

## Call to Action

Readers are encouraged to stay informed on international events by subscribing to a reliable news source. Additionally, they are prompted to engage in discussions about global politics and diplomatic relations on social media platforms to foster a deeper understanding of the complexities of international relations.

In conclusion, the Trump-Zelenskyy Oval Office meeting on February 28, 2025, was a pivotal moment in US-Ukraine relations and global diplomacy. By analyzing the key points discussed during the meeting and its implications on international politics, we gain valuable insights into the dynamics of diplomatic engagements at the highest levels of government.